In [4]:
import praw
import pandas as pd
import datetime
import json

# 初始化Reddit实例
reddit = praw.Reddit(
    client_id="84j0Vg5dY1skOG3wyGIbiA",
    client_secret="8fAD_w4V9X1BUD0_8Y-RcEjjVlacXQ",
    user_agent="RR Analytics/1.0 (by /u/Eric_Xu1025)"
)

# 获取子论坛
subreddit = reddit.subreddit("RajasthanRoyals")

# 收集帖子数据，包含post_id
posts_data = []
for post in subreddit.new(limit=10000):
    posts_data.append({
        "post_id": post.id,  # 新增post_id字段
        "title": post.title,
        "author": post.author.name if post.author else "Unknown",
        "score": post.score,
        "comments_count": post.num_comments,
        "url": post.url,
        "text": post.selftext,
        "comments": []  # 初始化空评论列表
    })

# 创建字典以便通过post_id快速查找帖子
posts_dict = {post["post_id"]: post for post in posts_data}

# 收集评论并关联到对应帖子
for post in subreddit.new(limit=10000):  # 可根据需要调整范围（如遍历posts_dict中的帖子）
    post_id = post.id
    if post_id not in posts_dict:
        continue  # 跳过未在初始搜索中的帖子
    post.comments.replace_more(limit=0)  # 加载所有评论
    for comment in post.comments.list():
        comment_data = {
            "comment_author": comment.author.name if comment.author else "Anonymous",
            "comment_body": comment.body,
            "comment_score": comment.score
        }
        posts_dict[post_id]["comments"].append(comment_data)

# 生成时间戳
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

# 保存为CSV（可选）
df_posts = pd.DataFrame(posts_data)
df_posts.to_csv(f"rr_posts_with_comments_{timestamp}.csv", index=False)

# 保存为JSON（结构化更清晰）
with open(f"rr_posts_with_comments_{timestamp}.json", "w", encoding="utf-8") as f:
    json.dump(posts_data, f, ensure_ascii=False, indent=4)

# 打印示例数据（验证结构）
print("示例帖子数据（含评论）:")
print({
    "post_id": posts_data[0]["post_id"],
    "title": posts_data[0]["title"],
    "comments_count": len(posts_data[0]["comments"])
})

示例帖子数据（含评论）:
{'post_id': '1jiowpf', 'title': 'WTF is these memes', 'comments_count': 1}


In [1]:
import praw
import pandas as pd
import datetime
import json
import time

# Initialize Reddit instance
reddit = praw.Reddit(
    client_id="84j0Vg5dY1skOG3wyGIbiA",
    client_secret="8fAD_w4V9X1BUD0_8Y-RcEjjVlacXQ",
    user_agent="RR Analytics/1.0 (by /u/Eric_Xu1025)"
)

# Configuration
SUBREDDIT_NAME = "RajasthanRoyals"
TOTAL_POSTS = 10000
SAVE_INTERVAL = 1000  # Save data every 1000 posts
REQUEST_DELAY = 2     # Delay between API requests (seconds)

def main():
    # Phase 1: Collect post IDs first
    print("[Phase 1/3] Collecting post IDs...")
    post_ids = []
    
    try:
        for post in reddit.subreddit(SUBREDDIT_NAME).new(limit=TOTAL_POSTS):
            post_ids.append(post.id)
            time.sleep(0.5)  # Basic rate limiting
            
            # Progress tracking
            if len(post_ids) % 100 == 0:
                print(f"Collected {len(post_ids)} post IDs", end="\r")
                
        print(f"\nTotal post IDs collected: {len(post_ids)}")
    
    except Exception as e:
        print(f"Error collecting post IDs: {str(e)}")
        return

    # Phase 2: Collect post details with comments
    print("\n[Phase 2/3] Collecting post details...")
    all_data = []
    comment_counter = 0
    
    for idx, post_id in enumerate(post_ids, 1):
        try:
            # Get post object directly by ID
            submission = reddit.submission(id=post_id)
            
            # Build post data structure
            post_data = {
                "post_id": submission.id,
                "title": submission.title,
                "author": submission.author.name if submission.author else "Unknown",
                "score": submission.score,
                "comments_count": submission.num_comments,
                "url": submission.url,
                "text": submission.selftext,
                "comments": []
            }
            
            # Process comments (with pagination handling)
            submission.comments.replace_more(limit=None)
            for comment in submission.comments.list():
                post_data["comments"].append({
                    "comment_id": comment.id,
                    "author": comment.author.name if comment.author else "Anonymous",
                    "body": comment.body,
                    "score": comment.score,
                    "created_utc": comment.created_utc
                })
                comment_counter += 1
            
            all_data.append(post_data)
            
            # Progress tracking
            print(f"Processed {idx}/{len(post_ids)} posts | Comments: {comment_counter}", end="\r")
            
            # Memory management: Save periodically
            if idx % SAVE_INTERVAL == 0:
                save_data(all_data, idx)
                all_data = []  # Clear memory
                
            # Rate limiting
            time.sleep(REQUEST_DELAY)
            
        except Exception as e:
            print(f"\nError processing post {post_id}: {str(e)}")
            continue
    
    # Final save
    if all_data:
        save_data(all_data, "final")
    
    # Phase 3: Consolidate data
    print("\n[Phase 3/3] Consolidating data files...")
    consolidate_results()
    
def save_data(data, batch_id):
    """Save data to timestamped JSON file"""
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"rr_data_{timestamp}_batch_{batch_id}.json"
    
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    
    print(f"\nSaved batch {batch_id} to {filename}")

def consolidate_results():
    """Combine all batch files into single dataset"""
    # Implementation logic for combining files
    # (Can be implemented based on specific needs)
    print("Data consolidation logic can be implemented here")

if __name__ == "__main__":
    main()
    print("\nOperation completed successfully!")

Version 7.7.1 of praw is outdated. Version 7.8.1 was released Friday October 25, 2024.


[Phase 1/3] Collecting post IDs...
Collected 900 post IDs
Total post IDs collected: 961

[Phase 2/3] Collecting post details...
Processed 961/961 posts | Comments: 11138
Saved batch final to rr_data_20250325_063610_batch_final.json

[Phase 3/3] Consolidating data files...
Data consolidation logic can be implemented here

Operation completed successfully!
